In [ ]:
# 모든 파일 리스팅
import pandas as pd
from glob import glob
from pathlib import Path
from tqdm import tqdm
import random

# ai hub 의 데이터는 train 과 valid 로 나누어져 있다.
# 동일한 사람이 두 셋에 동시에 나오지 않는다.
train_imgs = glob('train_imgs/*.png')
test_imgs = glob('val_imgs/*.png')

# 이미지 갯수를 출력해 본다.
len(train_imgs), len(test_imgs)

In [ ]:
# ai hub 의 train 이미지를 train/val로 쓰고
#           val 이미지를 test로 쓴다.
def to_label(imgs, phase):
    label = [[Path(e).stem, e] for e in imgs]
    label = [s.split('_') + [n] for s, n in label]
    df = pd.DataFrame(label, columns=['INOUT', 'PID', 'SN', 'D1', 'D2', 'NAME'])
    df['K_SN'] = df['PID'] + '_' + df['SN']
    df['K_D1'] = df['K_SN'] + '_' + df['D1']
    df['K_D2'] = df['K_D1'] + '_' + df['D2'] 
    df['CLS_NAME'] = df['K_SN']
    df['PHASE'] = p

In [ ]:
# train 과 test를 DataFrame 포멧으로 만든다. 
df_train = to_label(train_imgs, 'train')
df_test = to_label(test_imgs, 'test')

# 하나의 DataFrame 으로 통합한다.
df_total = pd.concat([df_train, df_test])

# 통합된 데이터 프레임을 출력해 본다
df_total

In [ ]:
# 통계 정보를 출력해 본다
df_total[['SN','PID','K_SN', 'K_D1', 'K_D2', 'CLS_NAME']].describe()

In [ ]:
#클래스를 인덱스화 한다.
kls_idx = {e:i for i, e in enumerate(sorted(list(set(df_total['CLS_NAME'].unique()))))}
df_total['KLS_IDX'] = df_total['CLS_NAME'].map(kls_idx) 
df_total


In [ ]:
# 무작위로 섞은 후 클래스 인덱스로 그룹화 한다.
df_train_g = df_total.query('PHASE != "test"').sample(frac=1.0).groupby('KLS_IDX')

# 각 클래스에서 맨 마지막 30개 이미지를 val 셋으로 옮긴다.
df_trains, df_vals = [], []
for KLS_IDX, df_i in df_train_g:
    df_trains.append(df_i.iloc[:-30])
    df_vals.append(df_i.iloc[-30:])